In [2]:
import requests
from bs4 import BeautifulSoup
import string
import random
import json
import re
from requests_html import HTMLSession
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
# function to get random urls
def links(num):
    link = []
    while True:
        query = 'https://www.youtube.com/results?search_query='
        url = query + ''.join(random.choice(string.ascii_lowercase) for _ in range(3))
        try:
            session = HTMLSession()
            response = session.get(url)
        except requests.exceptions.RequestException as e:
            print(e)
        text = response.html.html
        if len(set(link)) <= num:
            link.extend(re.findall('watch\?v=(.{11})',text))
        else:
            break
    return list(set(link))[:num]

In [4]:
urls = links(500)
features = ['link','uploaded_date','views','likes','dislikes']
youtube_data = pd.DataFrame(columns=features)
base_url = 'https://www.youtube.com/watch?v='

In [5]:
for link in (urls):
    
    youtube_url = base_url+str(link)
    video = requests.get(youtube_url)
    soup = BeautifulSoup(video.text,'html.parser')
    data = soup.find('script',string=re.compile('ytInitialData'))
    json_text = data.text.split('};')[0].replace('window["ytInitialData"] =','').strip()+'}'
    json_data = json.loads(json_text)
    
    views = np.nan
    upload_date = np.nan
    likes = np.nan
    dislikes = np.nan
    
    try:
        views = json_data['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]['videoPrimaryInfoRenderer']['viewCount']['videoViewCountRenderer']['viewCount']['simpleText']
        upload_date = json_data['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]['videoPrimaryInfoRenderer']['dateText']['simpleText']
        likes = json_data['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]['videoPrimaryInfoRenderer']['videoActions']['menuRenderer']['topLevelButtons'][0]['toggleButtonRenderer']['defaultText']['accessibility']['accessibilityData']['label']
        dislikes = json_data['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]['videoPrimaryInfoRenderer']['videoActions']['menuRenderer']['topLevelButtons'][1]['toggleButtonRenderer']['defaultText']['accessibility']['accessibilityData']['label']
    except KeyError as error:
        continue

    youtube_data = youtube_data.append({'link':youtube_url,'uploaded_date':upload_date,'views':views,
                                   'likes':likes,'dislikes':dislikes},ignore_index=True)

In [6]:
youtube_data

,link,uploaded_date,views,likes,dislikes
0,https://www.youtube.com/watch?v=chtUIhEFZyc,4 Oct 2019,"1,816 views",39 likes,No dislikes
1,https://www.youtube.com/watch?v=QQ_3S-IQm38,15 Feb 2018,"22,735,119 views","240,727 likes","10,948 dislikes"
2,https://www.youtube.com/watch?v=bo-LxCpnD2U,17 May 2020,419 views,26 likes,No dislikes
3,https://www.youtube.com/watch?v=XVuNGKnsqUA,21 Jul 2016,"2,335,590 views","67,927 likes",521 dislikes
4,https://www.youtube.com/watch?v=eBRnDTDgKss,19 Sep 2020,837 views,16 likes,No dislikes
...,...,...,...,...,...
470,https://www.youtube.com/watch?v=RSEBBk3-c80,18 Feb 2020,"1,175 views",7 likes,No dislikes
471,https://www.youtube.com/watch?v=R0NpahenvAg,1 Jan 2016,"10,727,439 views","97,797 likes","6,916 dislikes"
472,https://www.youtube.com/watch?v=4trJQqH66sU,9 Sep 2020,"2,716 views",219 likes,1 dislike
473,https://www.youtube.com/watch?v=4etilyqPUiY,4 Apr 2017,"33,767 views",412 likes,31 dislikes


In [7]:
youtube_data['views'] = youtube_data['views'].apply(lambda x: re.sub('views','',x).strip())
youtube_data['likes'] = youtube_data['likes'].apply(lambda x: re.sub('likes','',x).strip())
youtube_data['dislikes'] = youtube_data['dislikes'].apply(lambda x: re.sub('dislikes','',x).strip())

In [8]:
youtube_data['views'] = youtube_data['views'].apply(lambda x: re.sub('No','None',x))
youtube_data['likes'] = youtube_data['likes'].apply(lambda x: re.sub('No','None',x))
youtube_data['dislikes'] = youtube_data['dislikes'].apply(lambda x: re.sub('No','None',x))

In [9]:
youtube_data

,link,uploaded_date,views,likes,dislikes
0,https://www.youtube.com/watch?v=chtUIhEFZyc,4 Oct 2019,"1,816",39,None
1,https://www.youtube.com/watch?v=QQ_3S-IQm38,15 Feb 2018,"22,735,119","240,727","10,948"
2,https://www.youtube.com/watch?v=bo-LxCpnD2U,17 May 2020,419,26,None
3,https://www.youtube.com/watch?v=XVuNGKnsqUA,21 Jul 2016,"2,335,590","67,927",521
4,https://www.youtube.com/watch?v=eBRnDTDgKss,19 Sep 2020,837,16,None
...,...,...,...,...,...
470,https://www.youtube.com/watch?v=RSEBBk3-c80,18 Feb 2020,"1,175",7,None
471,https://www.youtube.com/watch?v=R0NpahenvAg,1 Jan 2016,"10,727,439","97,797","6,916"
472,https://www.youtube.com/watch?v=4trJQqH66sU,9 Sep 2020,"2,716",219,1 dislike
473,https://www.youtube.com/watch?v=4etilyqPUiY,4 Apr 2017,"33,767",412,31


In [10]:
youtube_data.to_csv('Youtube_scrape.csv')